# Preparando o ambiente

Instalação da bibliotecas necessárias

In [1]:
%pip install pandas
#substituir o '!' por '%' para Linux

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Importação das bibliotecas

In [2]:
from source.xml import read_xml
from source.fasta import generate_fasta
from source.utils import lista_de_arquivos, Setup_filesystem
import pandas as pd

In [3]:
Setup_filesystem()

Diretório data/raw já existe!
Diretório data/processed já existe!
Diretório data/processed/fasta_files já existe!


# Extração de dados

In [4]:
file_list = lista_de_arquivos("data/raw") # Importando a lista de todos os arquivos

* Adicionar no diretório  *data/raw*  o(s) arquivo(s) *.xml* em questão:

In [5]:
file_list_xml = [arquivo for arquivo in file_list if arquivo.endswith(".xml")] # Filtra a lista de arquivos para manter somente aquivos que seja xml

In [6]:
df = pd.DataFrame()
for arquivo_xml in file_list_xml:
    arquivo_xml = "data/raw/" + arquivo_xml
    df_temp = read_xml(path=arquivo_xml)
    df = pd.concat([df,df_temp])

# Transformando colunas e trabalhando especificamente com o pubmed
- A Pandas não gosta de espaços para o nome das colunas

In [7]:
# Renomeando a colunas Pubmed
df = df.rename(columns={'Creation date': 'Creation_date', 'Update date': 'Update_date', 'Pubmed accession':'Pubmed_number', 'Nucleotide Sequence':'Nucleotide_Sequence'})

In [8]:
df_filtrada = df[['Locus', 'Pubmed_number']]

In [9]:
df_explode = df_filtrada.explode(column='Pubmed_number').drop_duplicates()

In [10]:
for linha in df_explode['Pubmed_number']:
    print(type(linha))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [11]:
filtro = df_explode['Pubmed_number'] != ''
df_explode = df_explode[filtro]

In [12]:
df_explode = df_explode.query("Pubmed_number != ''") 

- Manter só as sequencias de genoma com mais de 200 nucleotídeos -> remover as vazias

In [13]:
df = df[df['Nucleotide_Sequence'] != '']

In [14]:
for coluna in df.columns:
    df[coluna] = df[coluna].apply(lambda x: x if x != ''  else 'N/A')

df['Pubmed_number'] = df['Pubmed_number'].apply(lambda x: x if x != [''] else ['N/A'])

# Gerando Arquivos FASTA e CSV

In [15]:
generate_fasta(df)

df.to_csv("data/processed/sequences.csv")
df_explode.to_csv("data/processed/pubmed_codes.csv")